In [13]:
import pandas as pd
import numpy as np
import os
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import layers
from keras import models
from keras import optimizers
import tensorflow as tf

In [123]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.is_gpu_available()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12984232663217635098
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2209624883
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12716201056627842778
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


True

In [124]:
data = pd.read_csv("D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train.csv")
print(data.iloc[0][0])
print(data.iloc[0][1])

000c1434d8d7
2


# Place Images Into Respective Class Folders

In [4]:
images = os.listdir("D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train_images")

# Eliminate the .png extension
for i in range(len(images)):
    images[i] = images[i][:-4]
    
    

for i in range(len(images)):
    imgName = data.iloc[i][0]
    diagnosis = data.iloc[i][1]
    
    source = 'D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train_images\\' + imgName + '.png'
    destination = 'D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train_images\\' + str(diagnosis) + '\\' + imgName + '.png'
    
    os.rename(source,destination)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train_images\\000c1434d8d7.png' -> 'D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train_images\\2\\000c1434d8d7.png'

# Train/Val Split

In [5]:
batch_size = 32
targetImageSize = 150

imageGen = ImageDataGenerator(rescale = 1./255,
                              validation_split = .25)
trainGen = imageGen.flow_from_directory(directory = "D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train_images",
                                       class_mode = "categorical",
                                       batch_size = batch_size,
                                       subset = "training",
                                       target_size = (targetImageSize,targetImageSize))

valGen = imageGen.flow_from_directory(directory = "D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\train_images",
                                       class_mode = "categorical",
                                       batch_size = batch_size,
                                       subset = "validation",
                                       target_size = (targetImageSize,targetImageSize))

Found 2749 images belonging to 5 classes.
Found 913 images belonging to 5 classes.


In [6]:
model = models.Sequential()

model.add(layers.Conv2D(32,(3,3) , activation = 'relu',
                       input_shape = (targetImageSize,targetImageSize,3)))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Conv2D(64 , (3,3) , activation = 'relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Conv2D(128 , (3,3) , activation = 'relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation = 'relu'))
model.add(layers.Dense(5 , activation = 'softmax'))

model.summary()

W0701 12:05:09.799506  4820 deprecation_wrapper.py:119] From c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0701 12:05:09.814491  4820 deprecation_wrapper.py:119] From c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0701 12:05:09.817470  4820 deprecation_wrapper.py:119] From c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0701 12:05:09.833415  4820 deprecation_wrapper.py:119] From c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Pleas

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
__________

In [7]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = optimizers.RMSprop(lr=1e-4),
             metrics = ['acc'])

W0701 12:05:38.242791  4820 deprecation_wrapper.py:119] From c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0701 12:05:38.250769  4820 deprecation_wrapper.py:119] From c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [10]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
history = model.fit_generator(trainGen,
                             steps_per_epoch = 85,
                             epochs = 20,
                             validation_data = valGen,
                             validation_steps = 28,
                             verbose = 1)

Epoch 1/20
85/85 [==============================] - 288s 3s/step - loss: 0.7465 - acc: 0.7316 - val_loss: 0.7472 - val_acc: 0.7196
Epoch 2/20
85/85 [==============================] - 252s 3s/step - loss: 0.7580 - acc: 0.7208 - val_loss: 0.7764 - val_acc: 0.7128
Epoch 3/20
85/85 [==============================] - 264s 3s/step - loss: 0.7296 - acc: 0.7312 - val_loss: 0.7322 - val_acc: 0.7412
Epoch 4/20
85/85 [==============================] - 280s 3s/step - loss: 0.7384 - acc: 0.7307 - val_loss: 0.7642 - val_acc: 0.7219
Epoch 5/20
85/85 [==============================] - 258s 3s/step - loss: 0.7243 - acc: 0.7299 - val_loss: 0.7267 - val_acc: 0.7401
Epoch 6/20
85/85 [==============================] - 271s 3s/step - loss: 0.6901 - acc: 0.7508 - val_loss: 0.7327 - val_acc: 0.7333
Epoch 7/20
85/85 [==============================] - 259s 3s/step - loss: 0.6988 - acc: 0.7422 - val_loss: 0.7174 - val_acc: 0.7378
Epoch 8/20
85/85 [==============================] - 261s 3s/step - loss: 0.6845 - a

In [11]:
model.save("m.hdf5")

In [37]:
img_path = "D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\test_images\\0a2b5e1a0be8.png"

img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

pred = model.predict(img_tensor)
np.where(pred == np.amax(pred[0]))[1][0]

2

In [117]:
testImages = os.listdir("D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\test_images")
testImagesNames = pd.read_csv("D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\test.csv")
testImagesNames = np.array(testImagesNames)
predArray = np.empty((len(testImagesNames),2),dtype='object')

for i in range(len(testImagesNames)-1):
    imgName = testImagesNames[i][0]
    imgPath = "D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\test_images\\" + imgName + ".png"
    img = image.load_img(imgPath,target_size=(150,150))
    imgTensor = image.img_to_array(img)
    imgTensor = np.expand_dims(imgTensor,axis=0)
    imgTensor /= 255.
    
    pred = model.predict(imgTensor)
    classification = np.where(pred == np.amax(pred[0]))[1][0]
    
    predArray[i][0] = imgName
    predArray[i][1] = str(classification)
    
    if(i%100 == 0):
        print("Completed:" + str(i))

Completed:0
Completed:100
Completed:200
Completed:300
Completed:400
Completed:500
Completed:600
Completed:700
Completed:800
Completed:900
Completed:1000
Completed:1100
Completed:1200
Completed:1300
Completed:1400
Completed:1500
Completed:1600
Completed:1700
Completed:1800
Completed:1900


In [118]:
predArray

array([['0005cfc8afb6', '2'],
       ['003f0afdcd15', '3'],
       ['006efc72b638', '2'],
       ...,
       ['ff64897ac0d8', '2'],
       ['ffa73465b705', '3'],
       [None, None]], dtype=object)

In [122]:
predArrayPD = pd.DataFrame(predArray)
predArrayPD.to_csv("D:\\School\\Summer_2019\\Kaggle_BlindnessCompetition\\aptos2019-blindness-detection\\predictions.csv")